# Data Clean withing Python 

In [ ]:
import pandas as pd

df = pd.read_csv('Step2.csv')
target = pd.DataFrame(df[['date', 'id','event_meal','MEAL', 'occasion_tagged', 'event_tagged']])
print("------------- Menu.csv ------------------")
print("before editing")
print(target.shape[0])
target = target[(target['date'] > '1840-01-01') & (target['date'] < '2018-12-01')]
print("after delete invalid date:")
print(target.shape[0])
target['meals'] = 'UNKNOWN'
target['occasion'] = 'UNKNOWN'
for index, row in target.iterrows():
    if row['event_meal'] != 'UNKNOWN':
        target.loc[index, 'meals'] = row['event_meal']
    else:
        if row['MEAL'] != 'UNKNOWN':
            target.loc[index, 'meals'] = row['MEAL']

    if row['occasion_tagged'] != 'UNKNOWN':
        target.loc[index, 'occasion'] = row['occasion_tagged']
    else:
        if row['event_tagged'] != 'UNKNOWN':
            target.loc[index, 'occasion'] = row['event_tagged']
target = target[['date', 'id','meals','occasion']]
target.to_csv('newMenu.csv')
print("final output:")
print(target.shape[0])

df = pd.read_csv('NYPL-Menus/MenuItem.csv')
print("------------- MenuItem.csv ------------------")
print("before editing")
print(df.shape[0])
df = df[(df['price'] > 0) & (df['price'] < 125000)]
print("after drop invalid price:")
print(df.shape[0])
target = pd.DataFrame(df[['price', 'id', 'menu_page_id','dish_id']])
target.to_csv('newMenuItem.csv')
print("final output:")
print(target.shape[0])

df = pd.read_csv('NYPL-Menus/Dish.csv')
print("------------- Dish.csv ------------------")
print("before editing")
print(df.shape[0])
df = df[['highest_price', 'lowest_price', 'id', 'name']]
target = df.dropna()
print("after drop null value:")
print(target.shape[0])
target.to_csv('newDish.csv')

# Develop Relational Database

In [ ]:
Menu = pd.read_csv('newMenu.csv')
Dish = pd.read_csv('newDish.csv')
MenuItem = pd.read_csv('newMenuItem.csv')
MenuPage = pd.read_csv('MenuPage.csv')

In [ ]:
Menu = Menu.loc[:, ~Menu.columns.str.contains('^Unnamed')]
Dish = Dish.loc[:, ~Dish.columns.str.contains('^Unnamed')]
MenuItem = MenuItem.loc[:, ~MenuItem.columns.str.contains('^Unnamed')]
MenuPage = MenuPage.loc[:, ~MenuPage.columns.str.contains('^Unnamed')]

## Create Database

In [ ]:
import sqlite3

In [ ]:
conn = sqlite3.connect('Full_database.db')
c = conn.cursor()

In [ ]:
# create dish table
c.execute('''create table Dish_Table(
    dish_id integer, 
    dish_name text,
    dish_lowest_price float,
    dish_highest_price float)''')

# create MenuItem table
c.execute('''create table MenuItem_Table(
    item_id integer,
    menu_page_id integer,
    dish_id integer,
    item_price float)''')

# create MenuPage table 
c.execute('''create table MenuPage_Table(
    menu_page_id integer,
    menu_id integer,
    page_number integer,
    page_image_id integer,
    page_full_height integer,
    page_full_width integer,
    page_uuid integer)''')

# create Menu table
c.execute('''create table Menu_Table(
    menu_id integer,
    occasion text,
    meals text,
    dated date)''')

## Fill Database 

In [ ]:
# dish data 
dish_var = Dish[['id', 'name', 'lowest_price', 'highest_price']]

dish_variable = [tuple(x) for x in dish_var.values]

# fill dish database
for row in dish_variable:
     c.execute('insert into Dish_Table values (?,?,?,?)', row)

print(pd.read_sql_query("SELECT * FROM Dish_Table", conn))

In [ ]:
# MenuItem data
item_var = MenuItem[['id', 'menu_page_id', 'dish_id', 'price']]

item_variable = [tuple(x) for x in item_var.values]

# fill menuItem database
for row in item_variable: 
    c.execute('insert into MenuItem_Table values (?,?,?,?)', row)

print(pd.read_sql_query("SELECT * FROM MenuItem_Table", conn))

In [ ]:
# MenuPage data
page_var = MenuPage[['id', 'menu_id', 'page_number', 'image_id', 'full_height', 'full_width', 'uuid']]

page_variable = [tuple(x) for x in page_var.values]

# fill MenuPage database
for row in page_variable: 
    c.execute('insert into MenuPage_Table values (?,?,?,?,?,?,?)', row)

print(pd.read_sql_query("SELECT * FROM MenuPage_Table", conn))

In [ ]:
# menu data
menu_var = Menu[['id', 'occasion', 'meals', 'date']]

menu_variable = [tuple(x) for x in menu_var.values]

# fill in menupage database
for row in menu_variable:
    c.execute('insert into Menu_Table values (?,?,?,?)', row)

print(pd.read_sql_query("SELECT * FROM Menu_Table", conn))

In [ ]:
conn.commit()

# Check logical Integrity Constraints

## Entity Integrity Constraints

In [ ]:
# menu_table
menu_null = pd.read_sql_query("select * from Menu_Table where menu_id is null", conn)
print(menu_null)

In [ ]:
# menuItem table
menuItem_null = pd.read_sql_query("select * from MenuItem_Table where item_id is null", conn)
print(menuItem_null)

In [ ]:
# MenuPage table
menuPage_null = pd.read_sql_query("select * from MenuPage_Table where menu_page_id is null", conn)
print(menuPage_null)

In [ ]:
# Dish table
dish_null = pd.read_sql_query("select * from Dish_Table where dish_id is null", conn)
print(dish_null)

##  Key Integrity Constraints 

In [ ]:
# Dish Table
print(pd.read_sql_query("select * from Dish_Table Group by dish_name, dish_lowest_price, dish_highest_price having count(*) > 1", conn))

In [ ]:
# MenuItem table
print(pd.read_sql_query("select * from MenuItem_Table group by menu_page_id, dish_id, item_price having count(*) > 1", conn)) 

In [ ]:
# MenuPage table
print(pd.read_sql_query("select * from MenuPage_Table Group by menu_id, page_image_id, page_full_height, page_full_width, page_uuid having count(*) > 1", conn))

In [ ]:
# Menu table
print(pd.read_sql_query("select * from Menu_Table Group by occasion, meals, dated having count(*) > 1", conn))

## Referenial Integrity Constraints

In [ ]:
# Menu page has to associate with at least one menu
menu_id_rem = pd.read_sql_query("select * from MenuPage_Table where MenuPage_Table.menu_id not in (select Menu_Table.menu_id from Menu_Table)", conn)

print('MenuPage_Table')
print('Menu_id')
print('------------------------')
print(menu_id_rem)

In [ ]:
# Menu Item has to associate with at least one menu page 
menuItem_id_rem = pd.read_sql_query("select * from MenuItem_Table where MenuItem_Table.menu_page_id not in (select MenuPage_Table.menu_page_id from MenuPage_Table)", conn)

print('MenuItem_Table')
print('menu_page_id')
print('------------------------')
print(menuItem_id_rem)

In [ ]:
# Dish in Menu item have exist in dish table
item_id_rem = pd.read_sql_query("select * from MenuItem_Table where MenuItem_Table.dish_id not in (select Dish_Table.dish_id from Dish_Table)", conn)

print('MenuItem_Table')
print('dish_id')
print('------------------------')
print(item_id_rem)

# Remove Records 

In [ ]:
# Delete records with no entity integrity constraints
# menu table
c.execute("Delete from Menu_Table where Menu_Table.menu_id is null")

In [ ]:
# Delete Records with no key integrity constraints
# dish table
c.execute("Delete from Dish_Table where Dish_Table.dish_id in (select dish_id from Dish_Table Group by dish_name, dish_lowest_price, dish_highest_price having count(*) > 1)")

# # MenuItem table
c.execute("Delete from MenuItem_Table where item_id not in (select distinct MenuItem_Table.item_id from MenuItem_Table group by menu_page_id, dish_id, item_price having count(*) = 1)")

# # MenuPage table
c.execute("Delete from MenuPage_Table where MenuPage_Table.menu_page_id in (select menu_page_id from MenuPage_Table Group by menu_id, page_image_id, page_full_height, page_full_width, page_uuid having count(*) > 1)")

# # Menu table
c.execute("Delete from Menu_Table where Menu_Table.menu_id not in (select menu_id from Menu_Table Group by occasion, meals, dated having count(*) = 1)")

In [ ]:
# Delete Records with no infereial integrity constraints 
# remove menu_id with no constraints in MenuPage table
c.execute("Delete from MenuPage_Table where MenuPage_Table.menu_id in (select distinct MenuPage_Table.menu_id from MenuPage_Table where MenuPage_Table.menu_id not in (select Menu_Table.menu_id from Menu_Table))")

# # remove menu_page_id with no constraints in MenuItem table
c.execute("Delete from MenuItem_Table where MenuItem_Table.menu_page_id in (select distinct MenuItem_Table.menu_page_id from MenuItem_Table where MenuItem_Table.menu_page_id not in (select MenuPage_Table.menu_page_id from MenuPage_Table))")

# # remove dish_id with no constraints from MenuPage table
c.execute("Delete from MenuItem_Table where MenuItem_Table.item_id in (select distinct MenuItem_Table.item_id from MenuItem_Table where MenuItem_Table.dish_id not in (select Dish_Table.dish_id from Dish_Table))")

In [ ]:
conn.commit()

# Results Analysis 

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

print(pd.read_sql_query("select dish_id, count(*) from MenuItem_Table group by dish_id order by count(dish_id) desc ", conn))

In [ ]:
# retrieve occasion for celery
celery_occasion = pd.read_sql_query("select Dish_Table.dish_name, MenuItem_Table.item_price, MenuPage_Table.page_number, occasion, dated from Menu_Table JOIN MenuPage_Table JOIN MenuItem_Table JOIN Dish_Table ON Menu_Table.menu_id = MenuPage_Table.menu_id AND MenuPage_Table.menu_page_id = MenuItem_Table.menu_page_id AND MenuItem_Table.dish_id = Dish_Table.dish_id WHERE Dish_Table.dish_id = 15.0 AND occasion != 'UNKNOWN'", conn)

celery_occasions = pd.DataFrame(data = celery_occasion.groupby('occasion')['dated'].count())
celery_occasions.plot(kind = 'bar')

In [ ]:
# retrieve meals for celery
celery_meals = pd.read_sql_query("select Dish_Table.dish_name, MenuItem_Table.item_price, MenuPage_Table.page_number, Menu_Table.meals, dated from Menu_Table JOIN MenuPage_Table JOIN MenuItem_Table JOIN Dish_Table ON Menu_Table.menu_id = MenuPage_Table.menu_id AND MenuPage_Table.menu_page_id = MenuItem_Table.menu_page_id AND MenuItem_Table.dish_id = Dish_Table.dish_id WHERE Dish_Table.dish_id = 15.0 AND meals != 'UNKNOWN'", conn)

Celery_meal = pd.DataFrame(data = celery_meals.groupby('meals')['dated'].count())
Celery_meal.plot(kind = 'bar')

In [ ]:
# retrieve price changes for celery over time 
celery_time = pd.read_sql_query("select Dish_Table.dish_name, MenuItem_Table.item_price, MenuPage_Table.page_number, dated from Menu_Table JOIN MenuPage_Table JOIN MenuItem_Table JOIN Dish_Table ON Menu_Table.menu_id = MenuPage_Table.menu_id AND MenuPage_Table.menu_page_id = MenuItem_Table.menu_page_id AND MenuItem_Table.dish_id = Dish_Table.dish_id WHERE Dish_Table.dish_id = 15.0", conn)

celery_time = pd.DataFrame(data = celery_time[['item_price', 'dated']])

celery_time = celery_time.set_index('dated')
celery_time.index = pd.to_datetime(celery_time.index)
celery_time.plot()